In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [2]:
from master import * 

/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Log path = /root/src/aleister/log/logging.log


In [3]:
model_id = 'test006'
model_name= 'cgm'
# model_name= 'slstm'


# args=['--symbol','BTC', '--execute_mode',"prepro",
#       '--model_id',model_id,'--model_name',model_name,
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

args=['--symbol','BTC', '--execute_mode',"train",
      '--model_id',model_id,'--model_name',model_name,
      '--config_source','ini',
      '--user',"Misaka00001", '--source','ipynb',
      "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
      '-train_sd','20210101','-train_ed','20210103',
      '-valid_sd','20210104','-valid_ed','20210105',
      '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"gtrain",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"deploy_model",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']


# args=['--symbol','BTC', '--execute_mode',"rpredict",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']



In [4]:
parser = make_parser()
arg_dict = vars(parser.parse_args(args))

# meta
_id = arg_dict["model_id"]
general_config_mode = arg_dict["general_config_mode"].upper()
private_api_mode = arg_dict["private_api_mode"].upper()
sym = arg_dict["symbol"]

model_name = arg_dict["model_name"].upper()

om = OperateMaster()
om.load_meta(_id, model_name, sym, general_config_mode, private_api_mode)
om.init_prepro()

# mlflow
mlflow_tags = {
    MLFLOW_USER: arg_dict["user"],
    MLFLOW_SOURCE_NAME: arg_dict["source"],
    MLFLOW_RUN_NAME: f"TRAIN_{dt.now().strftime('%y%m%d%H%M%s')}"
}
mlflow_client_kwargs = {
    "tracking_uri": os.environ['MLFLOW_TRACKING_URI']
}
om.set_mlflow_settings(mlflow_client_kwargs, mlflow_tags)

# load conofigs into each module
om.init_learning()


Log path = /root/src/aleister/log/logging.log
Log path = /root/src/aleister/log/logging.log


In [41]:

if arg_dict["execute_mode"] == "prepro":
    pass
    om.init_dataGen()
    # period
    train_start = arg_dict["train_start_date"] if "train_start_date" in arg_dict.keys() else None
    train_end = arg_dict["train_end_date"] if "train_end_date" in arg_dict.keys() else None
    valid_start = arg_dict["valid_start_date"] if "valid_start_date" in arg_dict.keys() else None
    valid_end = arg_dict["valid_end_date"] if "valid_end_date" in arg_dict.keys() else None
    test_start = arg_dict["test_start_date"] if "test_start_date" in arg_dict.keys() else None
    test_end = arg_dict["test_end_date"] if "test_end_date" in arg_dict.keys() else None
    # om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)
else:
    if arg_dict["execute_mode"] == "train":
        om.init_learning()
        # om.train()
    elif arg_dict["execute_mode"] == "gtrain":
        om.init_learning()
        # om.gtrain()
    elif arg_dict["execute_mode"] == "deploy_model":
        om.deploy_best_model()
    elif arg_dict["execute_mode"] == "rpredict":
        om.init_dataGen()
        # om.realtime_predict()


Log path = /root/src/aleister/log/logging.log


In [42]:
le = om.le
fp = om.fp
dg = om.dg

## Preprocessing

In [18]:
om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)

TypeError: 'dict_keys' object is not subscriptable

In [27]:
# gen data
fp._logger.info(f"{train_start}, {train_end}, {valid_start}, {valid_end}, {test_start}, {test_end}")
fetch_start = min([_date for _date in [train_start, train_end, valid_start, valid_end, test_start, test_end] if
                   _date is not None])
fetch_end = max([_date for _date in [train_start, train_end, valid_start, valid_end, test_start, test_end] if
                 _date is not None])

output_data_structure = fp.model_config.get("OUTPUT_DATA_STRUCTURE")
input_data_structure = fp.model_config.get("INPUT_DATA_STRUCTURE")
data_tree = dg.get_Xy(mode='train',
                           input_data_structure=input_data_structure,
                           output_data_structure=output_data_structure,
                           sym=sym, sd=fetch_start, ed=fetch_end)
fp._logger.info(f"[DONE] Get prepro raw datas. Sym={sym}, Data structure {fetch_start}~{fetch_end}")

# Split train/test/val ineach leaf
term_splited_data_tree = fp.split_child_by_period(data_tree, train_start, train_end, valid_start,
                                                       valid_end, test_start, test_end)
fp._logger.info("[DONE] Split by term.")



TypeError: 'dict_keys' object is not subscriptable

In [37]:
# Make each period tree
train_datas = fp.select_child_by_period(term_splited_data_tree, 'train')
test_datas = fp.select_child_by_period(term_splited_data_tree, 'test')
val_datas = fp.select_child_by_period(term_splited_data_tree, 'valid')
fp._logger.info("[DONE] picked by term.")

# to ndarray objects
train_datas = fp.get_input_structures(input_data_structure)(train_datas)
test_datas = fp.get_input_structures(input_data_structure)(test_datas, scalers=train_datas['scaler'])
val_datas = fp.get_input_structures(input_data_structure)(val_datas, scalers=train_datas['scaler'])
fp._logger.info("[DONE] To numpy and numpy ")

#  Note: If other data except  for X is using. Fix X_train": X_trains, "X_val":X_vals, "X_test": X_tests
fp.save_numpy_datas(**{
    "X_train": train_datas['data']['X'], "X_val": test_datas['data']['X'], "X_test": val_datas['data']['X'],
    "y_train": train_datas['data']['y'], "y_val": test_datas['data']['y'], "y_test": val_datas['data']['y'],
    "X_scaler": train_datas['scaler'], "note": {'input_dim': train_datas['input_dim']}
})

In [38]:
train_datas

{'data': {'X': array([[[2.79854995e-02, 3.47464128e-02, 2.80848856e-02, ...,
           5.84627415e-01, 8.91522668e-03, 0.00000000e+00],
          [1.25322605e-01, 1.26597718e-01, 1.47942651e-01, ...,
           4.78708085e-01, 0.00000000e+00, 7.69822941e-03]],
  
         [[3.49195744e-02, 3.46943111e-02, 3.50322438e-02, ...,
           2.52814518e-01, 1.74808366e-04, 0.00000000e+00],
          [1.26618782e-01, 1.26597718e-01, 1.44106835e-01, ...,
           4.55512281e-01, 0.00000000e+00, 6.15858353e-03]],
  
         [[1.75109825e-02, 1.73727429e-02, 0.00000000e+00, ...,
           7.06051037e-01, 0.00000000e+00, 4.50257065e-02],
          [1.18835362e-01, 1.18815593e-01, 1.37705434e-01, ...,
           4.06655338e-01, 0.00000000e+00, 3.84911470e-03]],
  
         ...,
  
         [[8.40291269e-01, 8.52288887e-01, 8.41946762e-01, ...,
           6.53904625e-01, 1.51034429e-02, 6.17794578e-03],
          [9.28392257e-01, 9.41090766e-01, 9.41645383e-01, ...,
           6.58897740e-01,

## Train 

In [44]:
obj_keys = ["X_train", "X_val", "y_train", "y_val", "note"]
X_trains, X_vals, y_trains, y_vals, note = fp.load_numpy_datas(obj_keys)
om.train_worker(X_trains, X_vals, y_trains, y_vals,  note )


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


TypeError: forward() missing 1 required positional argument: 'volume_data'

In [46]:
X_trains.shape

(3960, 2, 7)

In [34]:
predictions

array([[9.99979973e-01, 5.17890703e-06, 1.49594198e-05],
       [9.99979854e-01, 5.17760282e-06, 1.49675225e-05],
       [9.99979854e-01, 5.16807131e-06, 1.50279675e-05],
       ...,
       [1.07656269e-05, 3.91482763e-06, 9.99985337e-01],
       [1.08758677e-05, 3.94127801e-06, 9.99985218e-01],
       [9.99778450e-01, 1.91520576e-05, 2.02424315e-04]], dtype=float32)

## Gtrain

In [15]:
om.gtrain()

0


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



ValueError: need at least one array to concatenate

In [22]:
len(param_grid)

12

## train worker 

In [17]:


X_trains, X_vals, y_train, y_val, note = X_trains, X_vals, y_trains, y_vals, note
# train set up
model_params = {_k: le.hparams[_k] for _k in le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

le.get_model_instance(le.model_name, model_params)

fp.get_dataset_fn(le.hparams["dataset"])
dataset_params = {_k: le.hparams[_k] for _k in le.hparams["dataset_params"]}
train_loader, val_loader, _, _ = fp.get_dataloader(le.hparams["dataset"], X_trains, y_train, X_vals,
                                                        y_val, None, None, **dataset_params)

61


In [18]:
lossfn_params = {}
le.get_loss_fn(le.hparams["optimizer"], {})

optim_params = {
    'params': le.model.parameters(),
    'weight_decay': le.hparams["weight_decay"],
    'lr': le.hparams["lr"]
}

le.get_optimizer(le.hparams["loss_fn"], optim_params)

# train
le.train(train_loader, val_loader,
              batch_size=le.hparams["batch_size"], n_epochs=le.hparams["n_epoch"],
              n_features=1)

'8c83641bbc8948f88acc2793f9e22e1a'

## Train breakdown

In [37]:
batch_size=64
n_epochs=10
n_features=1
import numpy as np
import torch
from sklearn.metrics import accuracy_score

In [33]:
le._logger.info("[Start] Training. ID={0}".format(le.id))
# _ = le.get_model_save_path()  #todo : designate path

# setup
le.prediction = {}
le.predictions_out = {}
le.truths = {}
le.truths_out = {}
le.out_class = eval(le.model_config.get("OUT_CLASS"))

# mlflow
dict_config = {
    "batch_size": batch_size,
    "n_epochs": n_epochs,
    "optimizer": le.optimizer_name,
    "loss_fn": le.loss_fn_name
}

le.mlwriter.create_experiment(le.id, le.mlflow_tags)
le.mlwriter.log_params_from_omegaconf_dict(dict_config)

# start
for epoch in range(1, n_epochs + 1):
    batch_losses = []
    le.train_loader = train_loader
    for x_batchs, y_batchs in train_loader:
        le.optimizer.zero_grad()
        loss = le.train_step(x_batchs, y_batchs)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    le.train_losses.append(training_loss)
    le.mlwriter.log_metric('train_loss', training_loss, epoch)

    with torch.no_grad():
        batch_val_losses = []
        predictions = []
        truths = []
        for x_vals, y_val in val_loader:
            prediction, truth, val_loss = le.eval_step(x_vals, y_val)
            predictions.append(prediction)
            truths.append(truth)
            batch_val_losses.append(val_loss)

        validation_loss = np.mean(batch_val_losses)
        le.val_losses.append(validation_loss)
        
        predictions = np.concatenate(predictions)
        truths = np.concatenate(truths)

        # record
        le.prediction["val"], le.truths["val"] = predictions, truths
        predictions_out, truths_out = le.convert_model_value(predictions), le.convert_model_value(truths)
        le.predictions_out["val"], le.truths_out["val"] = predictions_out, truths_out
        acc = accuracy_score(truths_out, predictions_out)

        le.mlwriter.log_metric('valid_acc', acc, epoch)
        le.mlwriter.log_metric('valid_loss', validation_loss, epoch)
        # print(le.prediction["val"])
        # print(le.predictions_out["val"])

    if (epoch <= 10) | (epoch % 50 == 0):
        le._logger.info(
            f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
        )

# record last 
le.save_numpy_obj(predictions, "val_preds.csv")
le.save_numpy_obj(truths, "val_truth.csv")
le.save_numpy_obj(predictions_out, "val_preds_out.csv", type='int')
le.save_numpy_obj(truths_out, "val_truth_out.csv", type='int')

# le.save_model()
le.save_mlflow_model()
le._logger.info("[DONE] Training. ID={0}".format(le.id))


TypeError: forward() missing 2 required positional arguments: 'node_feature' and 'adj'

### Deploy 

In [7]:
om.deploy_best_model()

In [ ]:
list(om.le.postgres_ini['DOCKER'].keys())

['postgres_host',
 'postgres_port',
 'postgres_name',
 'postgres_username',
 'postgres_password',
 'postgres_servercursormode',
 'path',
 'hostname',
 'term',
 'base_dir',
 'kuloko_dir',
 'aleister_dir',
 'common_dir',
 'aiwass_dir',
 'mlflow_dir',
 'kuloko_ini',
 'aleister_ini',
 'kuloko_logdir',
 'aleister_logdir',
 'aiwass_logdir',
 'aiwass_log',
 'mongo_dir',
 'mlflow_host',
 'mlflow_tracking_uri',
 'mlflow_artifacts_uri',
 'default_local_file_and_artifact_path',
 'misaka_mq_host',
 'home',
 'lang',
 'language',
 'lc_all',
 'tz',
 'pydevd_use_frame_eval',
 'jpy_parent_pid',
 'clicolor',
 'pager',
 'git_pager',
 'mplbackend',
 'kmp_duplicate_lib_ok',
 'kmp_init_at_fork',
 'inidir']

In [7]:
om.init_postgres()

### Realtime prediction

In [8]:
realtime_datas, note = om.realtime_preprocessing()
now_dt = dl.dt_to_strYMDHM(dl.currentTime(om.le.tz))

# train set up
model_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

om.le.get_model_instance(om.le.model_name, model_params)

om.fp.get_dataset_fn(om.le.hparams["dataset"])
dataset_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["dataset_params"]}
_, _, _, test_loader_one = om.fp.get_dataloader(
    om.le.hparams["dataset"], None, None, None, None,
    realtime_datas["data"]["X"], realtime_datas["data"]["y"], **dataset_params)

prediction = om.le.predict(test_loader_one)

# Send mq
prediciton_dict = {"datetime": now_dt, "sym": om.sym, "data": prediction.tolist(),
                   "time": realtime_datas['times'], "name": om.model_name}
# prediciton_dict = {"datetime": now_dt, "sym": om.sym, "data": prediction.tolist()}
prediction_json = json.dumps(prediciton_dict )

om.init_realpred_mq()
om.mq_provider.connect_mq()
om.mq_provider.publish_mq(prediction_json)

# Save DB
if om.postgres_util is not None:
    try:
        om.postgres_util.insert_realtime_prediction(prediciton_dict)
    except Exception as e:
        om.le._logger.warning(f"[Failure] Save to postgres: {e}")


0
1


In [ ]:
om.

In [9]:
om.postgres_util.get_realtime_prediction("20220422","BTC")

[{'symbol': 'BTC',
  'datetime': datetime.datetime(2022, 4, 22, 20, 11, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))),
  'name': 'SLSTM',
  'value00': Decimal('0.27772006392478943'),
  'value01': Decimal('0.3690575063228607'),
  'value02': Decimal('0.3532223701477051'),
  'value03': None,
  'value04': None,
  'value05': None,
  'value06': None,
  'value07': None,
  'value08': None,
  'value09': None,
  'value10': None},
 {'symbol': 'BTC',
  'datetime': datetime.datetime(2022, 4, 22, 20, 12, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))),
  'name': 'SLSTM',
  'value00': Decimal('0.2777048945426941'),
  'value01': Decimal('0.369028240442276'),
  'value02': Decimal('0.3532669246196747'),
  'value03': None,
  'value04': None,
  'value05': None,
  'value06': None,
  'value07': None,
  'value08': None,
  'value09': None,
  'value10': None},
 {'symbol': 'BTC',
  'datetime': datetime.datetime(2022, 4, 22, 20, 13, tzinfo=datetime.timezone(datetime.timedelta(seconds=

# 

In [39]:
list(om.postgres_util.dao.config.values())

In [19]:
prediciton_dict

{'datetime': '202204222013',
 'sym': 'BTC',
 'data': [[0.27772006392478943, 0.3690575063228607, 0.3532223701477051],
  [0.2777048945426941, 0.369028240442276, 0.3532669246196747],
  [0.2777591943740845, 0.369108647108078, 0.3531321585178375]],
 'time': ['20220422201100000000',
  '20220422201200000000',
  '20220422201300000000']}

In [32]:
om.mq_provider.publish_mq(prediction_json)

In [37]:
om.init_postgres(om.le.general_config)
om.postgres_util.insert_realtime_prediction(prediciton_dict)

AttributeError: 'str' object has no attribute 'values'

In [38]:
prediciton_dict

{'datetime': '202204161704',
 'sym': 'BTC',
 'data': [[0.3904982805252075, 0.2895793914794922, 0.31992238759994507],
  [0.3903975486755371, 0.28962239623069763, 0.3199799656867981]]}

In [8]:
# scalers, note = om.fp.load_numpy_datas(["X_scaler", "note"])
# # Note: Avoid generator
# scaler_list = []
# om.fp._logger.info(f"{scalers}")
# for scaler in scalers:
#     scaler_list.append(scaler)

# output_data_structure = om.fp.model_config.get("OUTPUT_DATA_STRUCTURE")
# input_data_structure = om.fp.model_config.get("INPUT_DATA_STRUCTURE")

# # Fetch real data vim mq
# datas = om.dg.fetch_realdata()
# trades = datas["trade"]
# orderbooks = datas["orderbook"]
# om.fp._logger.info("[DONE] Get prepro raw data")

# # prepro
# data_tree = om.dg.get_Xy(mode="realtime",
#                            sym=om.sym,
#                            trades=trades,
#                            orderbooks=orderbooks,
#                            input_data_structure=input_data_structure,
#                            output_data_structure=output_data_structure)

# Split train/test/val ineach leaf
term_splited_data_tree = om.fp.split_child_by_period(data_tree, realtime=True)
om.fp._logger.info("[DONE] Split by term.")

# Make each period tree
realtime_datas = om.fp.select_child_by_period(term_splited_data_tree, 'realtime')
om.fp._logger.info("[DONE] picked by term.")

# to ndarray objects
realtime_datas = om.fp.get_input_structures(input_data_structure)(realtime_datas, scalers=scaler_list)

om.fp._logger.info("[DONE] To numpy and numpy ")


In [9]:
realtime_datas

{'data': {'X': array([[ 3.29605275e-04,  3.20806061e-04,  3.29605275e-04,
           3.20806061e-04, -2.52725316e-01,  6.78571429e-05,
           0.00000000e+00],
         [ 2.93404872e-04,  2.75819607e-04,  1.32751143e-04,
           1.70012782e-04,  1.53744869e+00,  4.11428571e-03,
           6.25000000e-05],
         [ 9.72375945e-05,  7.96523301e-05, -1.75859765e-05,
          -2.19816917e-05,  9.26402980e-01,  7.14285714e-06,
           8.90625000e-04],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00,  1.21428571e-04,
           0.00000000e+00]]),
  'y': [None]},
 'scaler': [MinMaxScaler()],
 'input_dim': 7,
 'times': ['20220418195300000000',
  '20220418195400000000',
  '20220418195500000000',
  '20220418195600000000']}

In [8]:
uri = om.load_prod_model()
om.le.load_mlflow_model(uri)

In [39]:
om.dg.init_mqclient("realtime")

In [40]:
realtime_datas, note = om.realtime_preprocessing()

# train set up
model_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

om.le.get_model_instance(om.le.model_name, model_params)

om.fp.get_dataset_fn(om.le.hparams["dataset"])
dataset_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["dataset_params"]}
_, _, _, test_loader_one = om.fp.get_dataloader(
    om.le.hparams["dataset"], None, None, None, None,
    realtime_datas["data"]["X"], realtime_datas["data"]["y"], **dataset_params)

prediction = om.le.predict(test_loader_one)

/usr/lib/python3.7/json/decoder.py:353: ResourceWarning:

unclosed <socket.socket fd=88, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 33834), raddr=('127.0.0.1', 36091)>

/usr/lib/python3.7/json/decoder.py:353: ResourceWarning:

unclosed <socket.socket fd=89, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 36091), raddr=('127.0.0.1', 33834)>

/usr/lib/python3.7/json/decoder.py:353: ResourceWarning:

unclosed <socket.socket fd=90, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.30.0.34', 36426)>



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [41]:
prediction

array([[0.3415096 , 0.30594435, 0.35254607],
       [0.34151688, 0.30586404, 0.35261902],
       [0.34152302, 0.3057909 , 0.35268608],
       [0.34153852, 0.30565593, 0.3528056 ],
       [0.3415452 , 0.30555186, 0.35290298],
       [0.34153417, 0.3053938 , 0.353072  ],
       [0.34127048, 0.3050306 , 0.35369885],
       [0.34033054, 0.3049156 , 0.3547539 ],
       [0.337725  , 0.3060506 , 0.35622436],
       [0.32880434, 0.3147323 , 0.3564633 ],
       [0.33037305, 0.31286868, 0.35675827],
       [0.32587644, 0.3180891 , 0.35603452],
       [0.328398  , 0.31463355, 0.35696852],
       [0.33168358, 0.31134278, 0.35697368],
       [0.32661828, 0.3174176 , 0.35596406],
       [0.3243381 , 0.3194604 , 0.35620144],
       [0.3264847 , 0.3164663 , 0.35704902],
       [0.32422766, 0.31926897, 0.35650343],
       [0.32788235, 0.3147693 , 0.35734838],
       [0.32715115, 0.31599855, 0.35685027],
       [0.32450613, 0.31907368, 0.3564202 ],
       [0.32345596, 0.31998175, 0.35656226],
       [0.

In [42]:
import numpy as np
import json


In [43]:
om.init_realpred_mq()

In [44]:
om.mq_provider.connect_mq()

In [45]:
om.mq_provider.publish_mq(prediction)

In [7]:
# todo: mode conider
uri = om.load_prod_model()
om.le.load_mlflow_model(uri)

realtime_datas, note = om.realtime_preprocessing()

In [8]:

# train set up
model_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

om.le.get_model_instance(om.le.model_name, model_params)

om.fp.get_dataset_fn(om.le.hparams["dataset"])
dataset_params = {_k: om.le.hparams[_k] for _k in om.le.hparams["dataset_params"]}

In [9]:
 realtime_datas["data"]["y"]

[None]

In [28]:


_, _, test_loader, test_loader_one = om.fp.get_dataloader(
    om.le.hparams["dataset"], None, None, None,None, 
    realtime_datas["data"]["X"], realtime_datas["data"]["y"], **dataset_params)

prediction = om.le.predict(test_loader_one)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [29]:
prediction

array([[0.3245815 , 0.39931172, 0.27610686],
       [0.32454658, 0.39937896, 0.2760745 ],
       [0.32437223, 0.39966905, 0.27595875],
       [0.3242796 , 0.39981595, 0.27590445],
       [0.3244523 , 0.39959255, 0.2759551 ],
       [0.3243367 , 0.39968795, 0.27597538],
       [0.32373467, 0.39987832, 0.276387  ],
       [0.32428795, 0.3999641 , 0.27574793],
       [0.32314497, 0.3986499 , 0.2782052 ],
       [0.31712675, 0.38409546, 0.2987778 ],
       [0.31790307, 0.3901374 , 0.29195955],
       [0.319827  , 0.39568365, 0.28448936],
       [0.31931797, 0.39416865, 0.28651342],
       [0.3206365 , 0.3969702 , 0.28239328],
       [0.32299575, 0.40116313, 0.27584112],
       [0.3189801 , 0.38995737, 0.29106253],
       [0.32102448, 0.3962355 , 0.28274003],
       [0.31982166, 0.3934258 , 0.28675255],
       [0.32250905, 0.39965972, 0.27783126],
       [0.3213373 , 0.3956371 , 0.28302556],
       [0.3206634 , 0.39435068, 0.28498593]], dtype=float32)

In [ ]:
import torch
data_loader=test_loader
with torch.no_grad():
    predictions = []
    truths = []
    le.test_loader = data_loader
    for x_tests, y_test in data_loader:
        prediction, truth, val_loss = le.eval_step(x_tests, y_test)
        predictions.append(prediction)
        truths.append(truth)

In [81]:
4//5

0

In [15]:
len( test_loader_one)

21

In [38]:
import pandas as pd
for _key in json_rst.keys():
    try:
        df = pd.DataFrame(json_rst[_key])
        print(1)
        df["datetime"] = df["time"].apply(lambda x: dl.strYMDHMSF_to_dt(x))
        del df["time"]
        df.set_index("datetime", inplace=True)
        dfs[_key] = df
        dg._logger.info(f"[DONE] Load data. Key={_key}")
    except Exception as e:
        dg._logger.warning(f"[Failure] Cannot load data. Key={_key}:{e}",exc_info=True)
        

1
1


In [39]:
dfs

{'orderbook':                         symbol      bids0  bids0_size      bids1  bids1_size  \
 datetime                                                                       
 2022-04-01 18:20:13.846    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:14.142    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:14.856    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:15.366    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:15.872    BTC  5524214.0      0.0002  5524200.0       0.001   
 ...                        ...        ...         ...        ...         ...   
 2022-04-01 18:20:59.421    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:20:59.912    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:21:00.418    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:21:00.924    BTC  5524214.0      0.0002  5524200.0       0.001   
 2022-04-01 18:

In [27]:
from util import daylib
from util import utils
dl = daylib.daylib()

In [ ]:
# prepro
X = self.dg.get_Xy(trades=trades, orderbooks=orderbooks, mode="realtime", method='flatten_v1')
X, _ = self.fp.feature_label_split(df=X, target_col=ans_col)
X, _ = self.fp.scalingX(X, scaself.ler)

In [44]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset

class SequenceDataset(Dataset):
    def __init__(self):
        super().__init__(1, 2, 3)
        # super().__init__(batch_size=1, drop_last=False)
        self.X_tensors = tensors[:-1]
        self.y_tensor = tensors[-1]
        self.window_size = params["window_size"]
        self.batch_size = params["batch_size"]  # keep
        self.Xs = [X_tensor.numpy() for X_tensor in self.X_tensors]

        if self.y_tensor is None:
            # Note; Xs[0] must have data length
            length = len(self.Xs[0])
            self.y = np.array([np.nan for _ in range(length)])
        else:
            self.y = self.y_tensor.numpy()

        if self.batch_size > len(self.y):
            self.batch_size = len(self.y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        print(idx)
        if idx >= (self.window_size - 1):
            idx_start = idx - self.window_size + 1
            Xs = []
            for X in self.Xs:
                Xs.append(X[idx_start:idx + 1:])
        else:
            n_padding = self.window_size - idx - 1
            Xs = []
            for X in self.Xs:
                padding = np.repeat(X[0][None, :], n_padding, axis=0)
                X_origin = X[:idx + 1]
                X_padded = np.vstack([X_origin, padding])
                Xs.append(X_padded)

        ys = self.y[idx]
        # print(ys)
        return Xs, ys


In [26]:

import pika
import uuid
import time

In [27]:
class Car:
    def __init__(self, make, model, year):
        self.make = make
        self.model = model
        self.year = year
        self.odometer_readings = 0

    def getname(self):
        longname = f'{self.year} {self.model} {self.year}'
        return longname.title()

    def read_odometer(self):
        print(f'This car has {self.odometer_readings} miles on it')

    def update_odometer(self, mileage):
        if mileage >= self.odometer_readings:
            self.odometer_readings = mileage
        else:
            print('You cant roll back  the odometer')

    def increment_odometer(self, mileage):
        self.odometer_readings += miles

class Battery:
    def __init__(self, battery_size = 75):
        self.battery_size = battery_size

    def describe_battery(self):
        print(f'This car has a {self.battery_size}-kWh battery in it')

    def get_range(self):
        if self.battery_size == 75:
            range = 260
        elif self.battery_size == 1000:
            range = 310
        print(f"This car can go {range} miles on a full charge")

class electriccar(Car):
    def __init__(self, make, model, year):
        super().__init__(make, model, year)
        self.battery = Battery()

namedtuple_TokenInfo:1: ResourceWarning:

unclosed <socket.socket fd=95, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.30.0.34', 34130)>

namedtuple_TokenInfo:1: ResourceWarning:

unclosed <socket.socket fd=93, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 60309), raddr=('127.0.0.1', 52910)>

namedtuple_TokenInfo:1: ResourceWarning:

unclosed <socket.socket fd=92, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 52910), raddr=('127.0.0.1', 60309)>

namedtuple_TokenInfo:1: ResourceWarning:

unclosed <socket.socket fd=106, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.30.0.34', 34174)>

namedtuple_TokenInfo:1: ResourceWarning:

unclosed <socket.socket fd=105, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 56983), raddr=('127.0.0.1', 42010)>

namedtuple_TokenInfo:1: ResourceWarning:

unclosed <socket.s

In [8]:
mqserver_host ="misaka_rabbitmq"
mqname = "aleister_real"
routing_key =  "aleister_real"
import pika, uuid, time

In [23]:
class RpcClient(object):

    def __init__(self, mqserver_host, mqname, routing_key):
        self.routing_key = routing_key
        self.mqserver_host = mqserver_host
        self.connect_mq()

        result = self.channel.queue_declare(queue=mqname, exclusive=False)
        self.callback_queue = result.method.queue

        self.channel.basic_consume(
            queue=self.callback_queue,
            on_message_callback=self.on_response,
            auto_ack=True)
        
    def connect_mq(self):
        print(
            f"[START] RPC: Connect MQ server... Host={self.mqserver_host}")
        self.connection = pika.BlockingConnection(
            pika.ConnectionParameters(host=self.mqserver_host))
        self.channel = self.connection.channel()
        print(
            f"[DONE] RPC: Connect MQ server... Host={self.mqserver_host}")


    def is_connect(self):
        try:
            if self.connection is None:
                return False
            return self.connection.is_open
        except Exception as e:
            print(
                f"Connection Check Failure... Host={self.mqserver_host}:{e}", exc_info=True)
            return False

    def close_mq(self):
        try:
            self.connection.close()
            self.channel = None
            print(
                f"[DONE] RPC: disconnect MQ server... Host={self.mqserver_host}")
        except:
            print(f"Already Closed.  RPC: disconnect MQ server... Host={self.mqserver_host}")

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            time.sleep(1)
            self.response = body

    def call(self, commannd_msg=None):
        commannd_msg = "" if commannd_msg is None else commannd_msg
        self.response = None
        self.corr_id = str(uuid.uuid4())
        print(f"[REQUEST] Call RPC. ID={self.corr_id}, Command={commannd_msg}, Roiting={self.routing_key}, CallBack to={self.callback_queue} ")
        self.channel.basic_publish(
            exchange='',
            routing_key=self.routing_key,
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
            ),
            body=str(commannd_msg))
        # Note:  anxious....
        while self.response is None:
            self.connection.process_data_events(time_limit=5)
            time.sleep(1)
            print(f"Waiting....")
        return self.response

    def end_call(self):
        self.call("END")


# fibonacci_rpc = FibonacciRpcClient()

# print(" [x] Requesting fib(30)")
# response = fibonacci_rpc.call(30)
# print(" [.] Got %r" % response)

/usr/local/lib/python3.7/dist-packages/IPython/core/compilerop.py:101: ResourceWarning:

unclosed <socket.socket fd=90, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.30.0.34', 38108)>

/usr/local/lib/python3.7/dist-packages/IPython/core/compilerop.py:101: ResourceWarning:

unclosed <socket.socket fd=89, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 44775), raddr=('127.0.0.1', 47654)>

/usr/local/lib/python3.7/dist-packages/IPython/core/compilerop.py:101: ResourceWarning:

unclosed <socket.socket fd=88, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 47654), raddr=('127.0.0.1', 44775)>



In [30]:
class RpcClient(object):

    def __init__(self, mqserver_host, mqname, routing_key, logger):
        self.routing_key = routing_key
        self.mqserver_host = mqserver_host
        self.logger = logger
        self.connect_mq()

        result = self.channel.queue_declare(queue=mqname, exclusive=False)
        self.callback_queue = result.method.queue

        self.channel.basic_consume(
            queue=self.callback_queue,
            on_message_callback=self.on_response,
            auto_ack=True)
        
    def connect_mq(self):
        print(
            f"[START] RPC: Connect MQ server... Host={self.mqserver_host}")
        self.connection = pika.BlockingConnection(
            pika.ConnectionParameters(host=self.mqserver_host))
        self.channel = self.connection.channel()
        print(
            f"[DONE] RPC: Connect MQ server... Host={self.mqserver_host}")


    def is_connect(self):
        try:
            if self.connection is None:
                return False
            return self.connection.is_open
        except Exception as e:
            self.logger.warning(
                f"Connection Check Failure... Host={self.mqserver_host}:{e}", exc_info=True)
            return False

    def close_mq(self):
        try:
            self.connection.close()
            self.channel = None
            self.logger.info(
                f"[DONE] RPC: disconnect MQ server... Host={self.mqserver_host}")
        except:
            self.logger.warning(f"Already Closed.  RPC: disconnect MQ server... Host={self.mqserver_host}")

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            time.sleep(1)
            self.response = body

    def call(self, commannd_msg=None):
        commannd_msg = "" if commannd_msg is None else commannd_msg
        self.response = None
        self.corr_id = str(uuid.uuid4())
        self.logger.info(
            f"[REQUEST] Call RPC. ID={self.corr_id}, Command={commannd_msg}, Roiting={self.routing_key}, CallBack to={self.callback_queue} ")
        self.channel.basic_publish(
            exchange='',
            routing_key=self.routing_key,
            properties=pika.BasicProperties(
                reply_to=self.callback_queue,
                correlation_id=self.corr_id,
            ),
            body=str(commannd_msg))
        # Note:  anxious....
        while self.response is None:
            self.connection.process_data_events(time_limit=5)
            time.sleep(1)
            self.logger.info(f"Waiting....")
        return self.response

    def end_call(self):
        self.call("END")


In [31]:

# rpc_client = RpcClient(mqserver_host, mqname, routing_key)
rpc_client = RpcClient(mqserver_host, mqname, routing_key, om.dg._logger)

[START] RPC: Connect MQ server... Host=misaka_rabbitmq
[DONE] RPC: Connect MQ server... Host=misaka_rabbitmq


In [32]:
rpc_client.is_connect()

True

In [33]:
rpc_client.call()

b'{"orderbook": [{"time": "20220416162318816000", "symbol": "BTC", "bids0": 5114080.0, "bids0_size": 0.1495, "bids1": 5114001.0, "bids1_size": 0.02, "bids2": 5114000.0, "bids2_size": 0.0014, "bids3": 5113996.0, "bids3_size": 0.02, "bids4": 5113600.0, "bids4_size": 0.002, "asks0": 5115500.0, "asks0_size": 2.0, "asks1": 5115650.0, "asks1_size": 0.5, "asks2": 5116250.0, "asks2_size": 0.1, "asks3": 5116350.0, "asks3_size": 0.4, "asks4": 5117000.0, "asks4_size": 2.0}, {"time": "20220416162319323000", "symbol": "BTC", "bids0": 5114080.0, "bids0_size": 0.1495, "bids1": 5114001.0, "bids1_size": 0.02, "bids2": 5114000.0, "bids2_size": 0.0014, "bids3": 5113996.0, "bids3_size": 0.02, "bids4": 5113600.0, "bids4_size": 0.002, "asks0": 5115500.0, "asks0_size": 2.0, "asks1": 5115650.0, "asks1_size": 0.5, "asks2": 5116100.0, "asks2_size": 0.1, "asks3": 5116200.0, "asks3_size": 0.4, "asks4": 5116800.0, "asks4_size": 2.0}, {"time": "20220416162319740000", "symbol": "BTC", "bids0": 5114080.0, "bids0_size

In [29]:
rpc_client.connection.close()